In [1]:
# Colab Setup (Run this first)

In [2]:
!pip install litebird_sim rich

# `litebird_sim` scanning and pointing


To run this notebook, you have several options:

-   If you are running this under Binder, you should already be set!
-   If you are running this under Google Colab, be sure to run the cell with `!pip install…` that is right above the title.
-   If you are running this locally, you should first create and activate a new virtual environment with the commands

    ```
    python -m venv ./my_venv
    source ./my_venv/bin/activate
    ```

    (you can use Conda environments, if you prefer) and install Jupyter and litebird_sim in it:

    ```
    pip install jupyter litebird_sim
    ```

    If you have a local copy of the `litebird_sim` repository cloned from <https://github.com/litebird/litebird_sim> (e.g., because you're part of the Simulation Team!), you can use a _development install_ instead:

    ```
    cd /my/local/copy/litebird_sim
    pip install -e .
    ```


## Set up the environment


In [3]:
# Using this file, we can use "import litebird_sim" even if it is not installed system-wide

We start by importing a few libraries that will be useful in this notebook.


In [4]:
import litebird_sim as lbs
import numpy as np
import astropy
import astropy.time

import inspect

%matplotlib inline

In [5]:
lbs.PTEP_IMO_LOCATION

PosixPath('/Users/luca/Documents/Universita/litebird/simteam/codes/litebird_sim/litebird_sim/default_imo/schema.json.gz')

## Parameters of the simulation

We will simulate one hour of the full 140 GHz LFT channel. Their definition will be taken from the LiteBIRD Instrument MOdel (IMO) version vPTEP (**new!**). See the [documentation](https://litebird-sim.readthedocs.io/en/latest/simulations.html#simulations) for more details about the input parameters.


In [6]:
telescope = "LFT"
channel = "L4-140"

start_time = astropy.time.Time("2025-01-01T00:00:00")
mission_time_hrs = 1

imo_version = "vPTEP"

# Resolution of the output maps
nside = 64

## Scanning strategy

To use the IMO bundled in `litebird_sim`, one needs to do the following:


In [7]:
# This is the folder where the final report with the results of the simulation will be saved
base_path = ".test"

imo = lbs.Imo(flatfile_location=lbs.PTEP_IMO_LOCATION)

# initializing the simulation
sim = lbs.Simulation(
    base_path=base_path,
    imo=imo,
    start_time=start_time,
    duration_s=mission_time_hrs * 3600.0,
    random_seed=12345,  # seed for the random number generator (MANDATORY parameter!!!)
)

The following command will take some time: it needs to compute the ephemerides of the Earth with respect to the Ecliptic reference frame and derive the orientation of the LFT instrument as a function of time for the whole duration of the simulation. This step will be needed later, when we will obtain the pointings for the detectors involved in the simulation. See the [documentation](https://litebird-sim.readthedocs.io/en/latest/scanning.html#scanning-strategy) for more details about the scanning strategy.


In [8]:
# Generate the quaternions describing how the instrument moves in the Ecliptic reference frame
sim.set_scanning_strategy(
    imo_url=f"/releases/{imo_version}/satellite/scanning_parameters/", delta_time_s=1
)
# The parameter `delta_time_s` specifies how often should quaternions be computed

Once the scanning strategy has been set the class [spin2ecliptic_quats](https://litebird-sim.readthedocs.io/en/master/scanning.html) contains the quaterions describing the scanning of the spin axis

In [9]:
print(getattr(sim.spin2ecliptic_quats, "quats").shape)
# ``(N_scanning_samples, 4)`` array

(3601, 4)


The following instructions load from the IMO the information about the [instrument](https://litebird-sim.readthedocs.io/en/latest/detectors.html#detectors-channels-and-instruments) and the [detectors](https://litebird-sim.readthedocs.io/en/latest/detectors.html#detectors-channels-and-instruments) used in the simulation.


In [10]:
# Load the definition of the instrument (MFT)
sim.set_instrument(
    lbs.InstrumentInfo.from_imo(
        imo,
        f"/releases/{imo_version}/satellite/{telescope}/instrument_info",
    )
)

When setting the instrument, the simulation acquires information about the spin-to-boresight rotation for the selected instrument, as well as any additional rotation of the boresight, if present

In [11]:
print(getattr(sim.instrument, "spin_boresight_angle_rad"))
print(getattr(sim.instrument, "boresight_rotangle_rad"))

0.8726646259971648
0.0


The spin to boresight angle can be easly customized initializing a new instrance of the instrument class

In [12]:
instrument = lbs.InstrumentInfo(
    name="Instrument_A", spin_boresight_angle_rad=np.deg2rad(40.0)
)
sim.set_instrument(instrument=instrument)

We are now ready to set the hwp. For this we have an abstract container, i.e. [HWP](https://litebird-sim.readthedocs.io/en/master/hwp.html#litebird_sim.hwp.HWP). 
Here we use the derved class [lbs.IdealHWP](https://litebird-sim.readthedocs.io/en/master/hwp.html#)

In [13]:
# Here we set the hwp speed
sim.set_hwp(
    lbs.IdealHWP(
        getattr(sim.instrument, "hwp_rpm") * 2 * np.pi / 60,
    ),  # sets the hwp
)

The final step in computing the pointings requires specifying a list of detectors. Each detector contributes the final rotation in the pointing chain, namely the transformation from the boresight to the detector reference frame

For getting the list of detectors in a Frequeny channel we can initialize the class [lbs.FreqChannelInfo](https://litebird-sim.readthedocs.io/en/master/detectors.html#litebird_sim.detectors.FreqChannelInfo) using the IMo.

In [ ]:
chinfo = lbs.FreqChannelInfo.from_imo(
    url=f"/releases/{imo_version}/satellite/{telescope}/{channel}/channel_info",
    imo=imo,
)

# filling dets with info and detquats with quaternions of the detectors in the channel
dets: list[lbs.DetectorInfo] = []
for n_det in chinfo.detector_names:
    det = lbs.DetectorInfo.from_imo(
        url=f"/releases/{imo_version}/satellite/{telescope}/{channel}/{n_det}/detector_info",
        imo=imo,
    )
    dets.append(det)

We are now ready to create a set of «[observations](https://litebird-sim.readthedocs.io/en/latest/observations.html#observations)» providing the list of detectors we have just created

In [15]:
# creating one observation
(obs,) = sim.create_observations(
    detectors=dets,
    n_blocks_det=1,
    n_blocks_time=1,  # blocks different from one if parallelizing
)

So, here comes the part where we need to simulate the pointings. Much of the work has already been done, as the ephemerides were already computed in the call to `sim.set_scanning_strategy` (see above), but this step is going to take its time too, because we are now deriving the pointings for **each** detector and store them in the list `pointings`.

The first step is done by [sim.prepare_pointings()](https://litebird-sim.readthedocs.io/en/master/simulations.html#litebird_sim.simulations.Simulation.prepare_pointings):


In [16]:
sim.prepare_pointings()

This method initializes a function in each observation that allows to compute the pointing on the fly 

In [17]:
pointing_matrix, hwp_angle = obs.get_pointings()

This returns two vectors

In [18]:
print(pointing_matrix.shape)  # [n_detectors, n_samples, 3]
# θ (colatitude, in radians), φ (longitude, in radians), and ψ (orientation angle, in radians)

print(hwp_angle.shape)  # [n_samples]

(144, 111600, 3)
(111600,)


get_pointings() supports three different syntaxes:
- Returns the pointing information for all the detectors with "get_pointings("all") or get_pointings()"

- Returns the pointing information for a list of detectors with something like "get_pointings([0,1])

- Returns the pointing information for one detector with something like "get_pointings(0)

In [19]:
# All the detectors are included
pointings, hwp_angle = obs.get_pointings("all")
# This returns ``(N_det, N_samples, 3)`` array
print(pointings.shape)

# Only the first and last detectors are included
pointings, hwp_angle = obs.get_pointings([0, 3])
# This returns ``(2, N_samples, 3)`` array
print(pointings.shape)

# Only the first detector is used
pointings, hwp_angle = obs.get_pointings(0)
# This returns ``(N_samples, 3)`` array
print(pointings.shape)

(144, 111600, 3)
(2, 111600, 3)
(111600, 3)


Pointing information can be precomputed and stored in the observations

In [20]:
sim.precompute_pointings()

Now all the observations have two new fields

In [21]:
print(obs.pointing_matrix.shape)
print(obs.hwp_angle.shape)

(144, 111600, 3)
(111600,)


The pointing type can be explicitly selected by specifying it in either get_pointings() or precompute_pointings()

In [22]:
pointings, hwp_angle = obs.get_pointings(0, pointings_dtype=np.float32)
print(pointings.dtype)

sim.precompute_pointings(pointings_dtype=np.float32)
print(obs.pointing_matrix.dtype)

float32
float32


A method is also provided to convert the pointing of a single detector on the fly into Galactic coordinates.

In [23]:
pointing_gal = lbs.rotate_coordinates_e2g(obs.pointing_matrix[0])

This allows both to scan or convolve a map and to return maps in Galactic coordinates. Rotation is always performed on-the-fly.
For example see the signatures for these functions:

In [24]:
print("fill_tods")
print("---------")
sig = inspect.signature(sim.fill_tods)
for param in sig.parameters.values():
    print(param)

print("")

print("convolve_sky")
print("---------")
sig = inspect.signature(sim.convolve_sky)
for param in sig.parameters.values():
    print(param)

print("")

print("make_binned_map")
print("---------")
sig = inspect.signature(sim.make_binned_map)
for param in sig.parameters.values():
    print(param)

fill_tods
---------
maps: Union[numpy.ndarray, Dict[str, numpy.ndarray], NoneType] = None
input_map_in_galactic: bool = True
component: str = 'tod'
interpolation: Optional[str] = ''
pointings_dtype=<class 'numpy.float64'>
append_to_report: bool = True

convolve_sky
---------
sky_alms: Union[litebird_sim.spherical_harmonics.SphericalHarmonics, Dict[str, litebird_sim.spherical_harmonics.SphericalHarmonics], NoneType] = None
beam_alms: Union[litebird_sim.spherical_harmonics.SphericalHarmonics, Dict[str, litebird_sim.spherical_harmonics.SphericalHarmonics], NoneType] = None
input_sky_alms_in_galactic: bool = True
convolution_params: Optional[litebird_sim.beam_convolution.BeamConvolutionParameters] = None
component: str = 'tod'
pointings_dtype=<class 'numpy.float64'>
nside_centering: Optional[int] = None
append_to_report: bool = True
nthreads: Optional[int] = None

make_binned_map
---------
nside: int
output_coordinate_system: litebird_sim.coordinates.CoordinateSystem = <CoordinateSystem.Ga